This is a barebones tutorial on how to use the BigCodeBench-Agent benchmark from the SmallBench test suite. In it, we'll evaluate a ReAct agent running with gpt-4o-mini on a few questions from the train split.

To learn more about [SmallBench](https://www.joshuapurtell.com/posts/smallbench/), look here. For BigCodeBench, look [here](https://t.co/0qQ1nLM1ue). For ReAct, [here](https://react-lm.github.io/).

In [1]:
!pip install smallbench==0.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.7 MB/s eta 0:00:00
  Using cached setuptools-74.1.1-py3-none-any.whl.metadata (6.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

Let's add credentials. Your OpenAI and Groq keys are necessary for the core LM and backup json parser utilities, respectively. Modal is one option you can use to containerize your code execution, and is more ergonomic in this notebook setting.

In [2]:
import os
from getpass import getpass

openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key
groq_api_key = getpass("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = groq_api_key
modal_token_id = getpass("Enter your Modal Token ID: ")
os.environ["MODAL_TOKEN_ID"] = modal_token_id
modal_token_secret = getpass("Enter your Modal Token Secret: ")
os.environ["MODAL_TOKEN_SECRET"] = modal_token_secret

language_model = "gpt-4o-mini-2024-07-18"

Enter your OpenAI API key: ··········
Enter your Groq API key: ··········
Enter your Modal Token ID: ··········
Enter your Modal Token Secret: ··········


In [3]:
from apropos.src.core.lms.helpers import LLM
from apropos.src.bench.bigcodebench.main import (
    BigCodeBench_Question,
    BigCodeBenchComplete_Benchmark,
)
from smallbench.baselines.agents.react import SimpleReActLanguageAgent
from smallbench.benchmarks.bcb_a.aci import BCBEngine, BCBAgentComputerInterface
from smallbench.benchmarks.bcb_a.bench import BCB_AgentBenchmark
import asyncio

In order to initialize our ReAct agent, we need to give it baseline information about the task it's completing and the environment it's acting in. You can tweak this and see if it helps performance, but here we're just going to give it the actions available to it and some simple guidelines for how to succeed at the BigCodeBench task.

In [4]:
def get_contexts_extremely_hacky_demo_only():
    example_question = BigCodeBenchComplete_Benchmark().train[0]
    backend = BCBEngine(example_question)
    aci = BCBAgentComputerInterface(backend)
    contexts = {
        "premise": "You are a software engineer",
        "setting_information": "You are working to solve a computer science problem. You will need to submit a solution to the problem, which will be tested against a suite of hidden unit tests.",
        "actions": {
            action.action_name: {
                "action_context": action.action_context,
                "action_arg_spec": action.action_arg_spec,
                "action_description": action.action_description,
            }
            for action in aci.actions
        },
        "objective": "Please complete the problem by drafting a solution, creating unit tests, improving the solution, and submitting the solution.",
        "constraints": "You will be given a code_prompt_for_answer, which contains imports and the function signature. Your solution must comprise code that can be appended to code_prompt_for_answer and run as a single script.",
    }
    return contexts

In [5]:
contexts = get_contexts_extremely_hacky_demo_only()
agent = SimpleReActLanguageAgent(lm=LLM(language_model), contexts=contexts)

Generating v0.1.0_hf split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Generating v0.1.1 split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Generating v0.1.2 split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Let's run our ReAct agent!

In [ ]:
question_indices_to_test = range(20)
split = "train"
agent_benchmark = BCB_AgentBenchmark(backend="modal")
agent_mean_success, total_experiment_cost = await agent_benchmark.score_agent(
    agent, split=split, indices=question_indices_to_test, verbose=True
)

Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
Spinning up agent...
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: edit_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_submission
- Action: add_unit_test
- Action: add_unit_test
- Action: add_unit_test
- Action: add_unit_test
- Action